In [26]:
import pandas as pd
import datetime
import re

In [25]:
def parse_file(filename):
    casts = []
    date_pattern = re.compile(r'\d{4}/\d{2}/\d{2} \d{2}:\d{2}:\d{2}')
    
    with open(filename, 'r') as file:
        lines = file.readlines()
        
        i = 0
        while i < len(lines):
            header = lines[i].strip()
            if date_pattern.match(header[:19]):
                parts = header.split(maxsplit=3)
                if len(parts) >= 3:
                    date_str = parts[0] + " " + parts[1]
                    cast_identifier = parts[2] + (" " + parts[3] if len(parts) == 4 else "")
                else:
                    print(f"Header format is incorrect: {header}")
                    raise ValueError("Header format is incorrect")
                
                try:
                    date = datetime.datetime.strptime(date_str, '%Y/%m/%d %H:%M:%S')
                except ValueError:
                    print(f"Date format is incorrect: {date_str}")
                    raise ValueError(f"Incorrect date format: {date_str}")
                
                depth_temp_data = []
                i += 1
                while i < len(lines) and not date_pattern.match(lines[i].strip()[:19]):
                    depth, temp = lines[i].strip().split()
                    depth_temp_data.append((-abs(int(depth)), float(temp)))  # Ensure depth is negative
                    i += 1
                
                casts.append((date, cast_identifier, depth_temp_data))
            else:
                i += 1
    
    return casts

In [27]:
def merge_and_sort_casts(file1, file2):
    casts1 = parse_file(file1)
    casts2 = parse_file(file2)
    
    all_casts = casts1 + casts2
    all_casts.sort(key=lambda x: x[0])
    
    return all_casts

def write_output(filename, casts):
    with open(filename, 'w') as file:
        for cast in casts:
            date_str = cast[0].strftime('%Y/%m/%d %H:%M:%S')
            cast_identifier = cast[1]
            file.write(f'{date_str} {cast_identifier}\n')
            for depth, temp in cast[2]:
                file.write(f'{depth}\t{temp}\n')
            file.write('\n')


In [29]:
# Usage
file1 = 't_prof_LS.dat'
file2 = 't_prof.dat'
output_file = 't_prof_file.dat'

casts = merge_and_sort_casts(file1, file2)
write_output(output_file, casts)